# Setting up a Ray cluster with SmartSim

## 1. Start the cluster
We set up a SmartSim experiment, which will handle the launch of the Ray cluster.

First we import the relevant modules.

In [1]:
import numpy as np
import time
import argparse
import os

from ray.tune.progress_reporter import JupyterNotebookReporter
import ray
from ray import tune
import ray.util

from smartsim import Experiment
from smartsim.ext.ray import RayCluster

NUM_WORKERS = 2
alloc=None
launcher='slurm'

In [2]:
exp = Experiment("ray-cluster", launcher=launcher)
cluster = RayCluster(name="ray-cluster", run_args={}, path='', ray_args={"num-cpus": 48},
                     launcher=launcher, workers=NUM_WORKERS, alloc=alloc, batch=True)

In [3]:
if cluster.batch:
    cluster.head_model.batch_settings.add_preamble( ["source ~/.bashrc", "conda activate smartsim"])
    if NUM_WORKERS:
        cluster.worker_model.batch_settings.add_preamble ( ["source ~/.bashrc", "conda activate smartsim"])

In [4]:
exp.generate(cluster, overwrite=True)

09:13:23 horizon SmartSim[9197] INFO Working in previously created experiment


In [5]:
exp.start(cluster, block=False, summary=False)

09:13:32 horizon SmartSim[9197] INFO Ray cluster launched on nodes: ['nid00000', 'nid00002', 'nid00001']


## 2. Start the ray driver script

In [11]:
#ray.client(cluster.head_model.address +":10001").connect()
#ray.redis_password=cluster._ray_password
job_config = ray.job_config.JobConfig(runtime_env={"redis_password": cluster._ray_password}, worker_env={"redis_password": cluster._ray_password})
#ray.util.connect(cluster.head_model.address+":10001")
#print(cluster.head_model.address)
#print("ray://"+cluster.head_model.address+":10001")
ray.init("ray://"+cluster.head_model.address+":10001", _redis_password=cluster._ray_password)
#ray.client("ray://"+cluster.head_model.address+":10001").connect()


RuntimeError: Unexpected keyword argument(s) for Ray Client: _redis_password

In [6]:
print('''This cluster consists of
    {} nodes in total
    {} CPU resources in total
'''.format(len(ray.nodes()), ray.cluster_resources()['CPU']))

This cluster consists of
    16 nodes in total
    768.0 CPU resources in total



In [7]:
tune.run(
    "PPO",
    stop={"episode_reward_max": 200},
    config={
        "framework": "torch",
        "env": "CartPole-v0",
        "num_gpus": 0,
        "lr": tune.grid_search(np.linspace (0.001, 0.01, 100).tolist()),
        "log_level": "ERROR",
    },
    local_dir="/lus/scratch/arigazzi/ray_local/",
    verbose=0,
    fail_fast=True,
    log_to_file=True,
)

2021-07-27 09:50:16,473	WARNING trial_runner.py:229 -- The maximum number of pending trials has been automatically set to the number of available cluster CPUs, which is high (844 CPUs/pending trials). If you're running an experiment with a large number of trials, this could lead to scheduling overhead. In this case, consider setting the `TUNE_MAX_PENDING_TRIALS_PG` environment variable to the desired maximum number of concurrent trials.
(pid=70525) 2021-07-27 09:50:23,517	INFO trainer.py:696 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=70529) 2021-07-27 09:50:23,531	INFO trainer.py:696 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=70533) 2021-07-27 09:50:23,514	INFO trainer.py:696 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=70545) 2021-07-27 09:50:23,510	INFO tr

In [9]:

tune.run(
    "PPO",
    stop={"episode_reward_max": 200},
    config={
        "framework": "torch",
        "env": "CartPole-v0",
    #    "num_gpus": 0,
        "lr": tune.grid_search(np.linspace (0.001, 0.01, 200).tolist()),
        "log_level": "ERROR",
    #    "num_cpus_per_worker": 1,
    },
    local_dir="/lus/scratch/arigazzi/ray_local/",
    verbose=1,
    #fail_fast=True,
    #log_to_file=True,
    #progress_reporter = JupyterNotebookReporter(True),
)

(pid=3481) == Status ==
(pid=3481) Memory usage on this node: 6.5/187.6 GiB
(pid=3481) Using FIFO scheduling algorithm.
(pid=3481) Resources requested: 3.0/336 CPUs, 0/0 GPUs, 0.0/381.81 GiB heap, 0.0/167.63 GiB objects
(pid=3481) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=3481) Number of trials: 200/200 (199 PENDING, 1 RUNNING)
(pid=3481) 
(pid=3481) 


(pid=29257) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=29257) Instructions for updating:
(pid=29257) non-resource variables are not supported in the long term
(pid=29227) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=29227) Instructions for updating:
(pid=29227) non-resource variables are not supported in the long term
(pid=29261) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is 

(pid=3481) == Status ==
(pid=3481) Memory usage on this node: 14.2/187.6 GiB
(pid=3481) Using FIFO scheduling algorithm.
(pid=3481) Resources requested: 336.0/336 CPUs, 0/0 GPUs, 0.0/381.81 GiB heap, 0.0/167.63 GiB objects
(pid=3481) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=3481) Number of trials: 200/200 (88 PENDING, 112 RUNNING)
(pid=3481) 
(pid=3481) 


(pid=36702) 2021-07-20 14:55:17,184	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=36707) 2021-07-20 14:55:17,184	INFO trainer.py:694 -- Current log_level is ERROR. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=29254) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=29254) Instructions for updating:
(pid=29254) non-resource variables are not supported in the long term
(pid=46436) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future 

(pid=3481) == Status ==
(pid=3481) Memory usage on this node: 31.6/187.6 GiB
(pid=3481) Using FIFO scheduling algorithm.
(pid=3481) Resources requested: 336.0/336 CPUs, 0/0 GPUs, 0.0/381.81 GiB heap, 0.0/167.63 GiB objects
(pid=3481) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=3481) Number of trials: 200/200 (88 PENDING, 112 RUNNING)
(pid=3481) 
(pid=3481) 


...

(pid=46294) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=46294) Instructions for updating:
(pid=46294) non-resource variables are not supported in the long term
(pid=46293) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=46293) Instructions for updating:
(pid=46293) non-resource variables are not supported in the long term
(pid=46334) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is 

(pid=3481) == Status ==
(pid=3481) Memory usage on this node: 23.7/187.6 GiB
(pid=3481) Using FIFO scheduling algorithm.
(pid=3481) Resources requested: 276.0/336 CPUs, 0/0 GPUs, 0.0/381.81 GiB heap, 0.0/167.63 GiB objects
(pid=3481) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=3481) Number of trials: 200/200 (92 RUNNING, 108 TERMINATED)
(pid=3481) 
(pid=3481) 


(pid=56736) 2021-07-20 14:56:31,936	INFO trainable.py:101 -- Trainable.setup took 10.682 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
(pid=26536) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=26536) Instructions for updating:
(pid=26536) non-resource variables are not supported in the long term
(pid=26607) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=26607) Instructions for updating:
(pid=26607) non-resource variables are not supported in the long term
(pid=58928) WARN

(pid=3481) == Status ==
(pid=3481) Memory usage on this node: 26.6/187.6 GiB
(pid=3481) Using FIFO scheduling algorithm.
(pid=3481) Resources requested: 261.0/336 CPUs, 0/0 GPUs, 0.0/381.81 GiB heap, 0.0/167.63 GiB objects
(pid=3481) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=3481) Number of trials: 200/200 (87 RUNNING, 113 TERMINATED)
(pid=3481) 
(pid=3481) 
(pid=3481) == Status ==
(pid=3481) Memory usage on this node: 26.6/187.6 GiB
(pid=3481) Using FIFO scheduling algorithm.
(pid=3481) Resources requested: 261.0/336 CPUs, 0/0 GPUs, 0.0/381.81 GiB heap, 0.0/167.63 GiB objects
(pid=3481) Result logdir: /lus/scratch/arigazzi/ray_local/PPO
(pid=3481) Number of trials: 200/200 (87 RUNNING, 113 TERMINATED)
(pid=3481) 
(pid=3481) 
(pid=3481) == Status ==
(pid=3481) Memory usage on this node: 25.9/187.6 GiB
(pid=3481) Using FIFO scheduling algorithm.
(pid=3481) Resources requested: 246.0/336 CPUs, 0/0 GPUs, 0.0/381.81 GiB heap, 0.0/167.63 GiB objects
(pid=3481) Result logdir: /

(pid=3481) 2021-07-20 14:57:18,575	INFO tune.py:549 -- Total run time: 127.53 seconds (127.05 seconds for the tuning loop).


## 3. Stop cluster and release allocation

In [6]:
if alloc:
    slurm.release_allocation(alloc)

14:15:33 nid00000 smartsim.launcher.slurm.slurm[49585] INFO Releasing allocation: 1461745
14:15:33 nid00000 smartsim.launcher.util.shell[49585] DEBUG Executing Popen cmd: /opt/slurm/20.11.5/bin/scancel 1461745
14:15:33 nid00000 smartsim.launcher.slurm.slurm[49585] INFO Successfully freed allocation 1461745
14:15:34 nid00000 smartsim.launcher.taskManager[49585] DEBUG Removing Task 49675
14:15:34 nid00000 smartsim.launcher.taskManager[49585] DEBUG Sleeping, no tasks to monitor


In [6]:
exp.stop(cluster)

11:31:07 nid00054 SmartSim[37275] INFO Stopping model workers with job name workers-CD42DFIIKH5D
11:31:07 nid00054 SmartSim[37275] INFO Stopping model head with job name head-CD42DC9I3OQO


In [10]:
analysis = tune.run(
    "PPO",
    stop={"episode_reward_max": 200},
    config={
        "framework": "torch",
        "env": "CartPole-v0",
        "lr": tune.grid_search([0.001, 0.002, 0.003, 0.004, 0.005]),
    },
    progress_reporter = JupyterNotebookReporter(True)
)

<IPython.core.display.HTML object>


(pid=110180) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=110180) Instructions for updating:
(pid=110180) non-resource variables are not supported in the long term
(pid=40586) WARNING:tensorflow:From /lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=40586) Instructions for updating:
(pid=40586) non-resource variables are not supported in the long term
(pid=110180) 2021-05-25 10:04:22,637	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=42854) WARNING:tensorflow:From /lus/scratch/ar

(pid=27027) 
(pid=27027) <IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


(pid=27027) 2021-05-25 10:04:54,258	INFO tune.py:549 -- Total run time: 34.48 seconds (34.15 seconds for the tuning loop).
(pid=42854) 2021-05-25 10:04:54,221	ERROR worker.py:382 -- SystemExit was raised from the worker
(pid=42854) Traceback (most recent call last):
(pid=42854)   File "python/ray/_raylet.pyx", line 495, in ray._raylet.execute_task
(pid=42854)   File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
(pid=42854)   File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
(pid=42854)   File "/lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
(pid=42854)     return method(__ray_actor, *args, **kwargs)
(pid=42854)   File "/lus/scratch/arigazzi/anaconda3/envs/smartsim/lib/python3.8/site-packages/ray/actor.py", line 1001, in __ray_terminate__
(pid=42854)     ray.actor.exit_actor()
(pid=42854)   File "/lus/scratch/arigazzi/anaconda3/envs/smart

'A'